In [1]:
import os
import openai
from PyPDF2 import PdfReader
import glob, os
import JD
API_KEY = "Your_API_KEY"
openai.api_key=API_KEY

In [2]:
llm_model = "gpt-4"

In [3]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0.0, model=llm_model)

In [4]:
template_string = """You are a helpful assistant.\
You will be provided with text of a resume, and job description delimited by triple backticks.
Assign a score from 0 to 10 based on the alignment between the core technology in the resume and the job title in the job description.\
Score 0 if there is no correspondence between the core technology in the resume and the job title specified in the job description.\
Scores 1 to 3 indicate a mismatch between the content of the resume and the requirements outlined in the job description.\
For partial alignment between the resume and job description, give a score within the range of 4 to 7.\
A strong match between the resume and job description warrants a score between 8 and 10.  
**Resume** : {raw_data}```\n
**Job description** : {job_description}```\n
{format_instructions}
You must follow the guidelines below enclosed in three hashes to answer the query \n
###
Do not return any additional information in the score key of the response other than what is mentioned in the score range.
Please do consider the skills in the resume in order to ensure they are related to the job description.

###
"""

In [5]:
def getresumes():
   os.chdir(os.getcwd()+"\\dataset\\java_resumes")
   pdfs = []
   for file in glob.glob("*.pdf"):
       print(file)
       pdfs.append(file)
    
   return pdfs

In [6]:
pdfs = getresumes()

Abdullah_-_1121841946 (1).pdf
Haider_Pasha_1121841946.pdf
Haseeb_Abdul_1121841946.pdf
Hasnain_Haider_1121841946.pdf
Muhammad_AbdurRahim_1121841946.pdf
Sikander_Mehmood_1121841946.pdf


In [7]:
all_docs = list()
for each_file in pdfs:
    file_directory = os.getcwd()+f"\\{each_file}"
    all_docs.append(PdfReader(file_directory))
all_docs 

In [8]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template=template_string)

In [9]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [25]:
name_schema = ResponseSchema(name="Name",
                             description="Name of the person from the given Resume text")
score_schema = ResponseSchema(name="Score",
                                      description="""Assign a score from 0 to 10 based on the alignment between the core technology in the resume and the job title in the job description.\
Score 0 if there is no correspondence between the core technology in the resume and the job title specified in the job description and there is no match of the main programming language.\
Scores 1 to 3 indicate a mismatch between the content of the resume and the requirements outlined in the job description.\
For partial alignment between the resume and job description, give a score within the range of 4 to 7.\
A strong match between the resume and job description warrants a score between 8 and 10.\
                                               """)
description_schema = ResponseSchema(name="Description",
                                    description="Please justify your scoring in detail. Give details of what part has the positive impact and what part has the negative impact on scoring the candidat")

hire_schema = ResponseSchema(name="Why should we hire him",
                                    description="why should we hire the candidate from the given resume, shortly give the reason.")

no_hire_schema = ResponseSchema(name="Why should we not hire him",
                                    description="why should we not hire the candidate from the given resume, shortly give the reason.")

response_schemas = [name_schema, 
                    score_schema,
                    description_schema,
                    hire_schema,
                    no_hire_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [26]:
all_resumes = []
for each_resume in all_docs:
    raw_data = str()
    for i, page in enumerate(each_resume.pages):
        content = page.extract_text()
        raw_data+=content
    to_be_sent_messages = prompt.format_messages(
                raw_data=raw_data,
                job_description=JD.jd,
                format_instructions=format_instructions
                )
    response = chat(to_be_sent_messages)
    all_resumes.append(output_parser.parse(response.content))
    # print(to_be_sent_messages[0].content)
     

In [27]:
for _ in all_resumes:
    print(_["Score"])
    print(_["Description"])

0
The resume provided does not contain any information about the candidate's skills, work experience, or education. Therefore, it is impossible to determine if the candidate's skills align with the job description for a Java/Java EE/SpringBoot application developer. The job description requires a candidate with experience in Java/J2EE, Spring, Struts, Apache Velocity templates, Hibernate, SQL, PostgreSQL, client-side scripting including JavaScript, JQuery, HTML5, and knowledge of UML and Design Patterns. Without any information on the candidate's skills or experience, a score of 0 is assigned.
8
Haider Pasha has a strong background in Java, Spring Boot, and RESTful services, which are key requirements for the job description provided. He has experience in Agile methodology, which is also a requirement for the job. He has worked with SQL databases and has a good understanding of microservices architecture, both of which are mentioned in the job description. However, he does not have exp

In [13]:
import json 
with open(os.getcwd().split("\\")[-1]+'with_Java_jd.json', "w") as file:
    json.dump(all_resumes, file)